# Exercise 5-3: Get data from a database

In [42]:
import pandas as pd

In [44]:
fires_url = 'https://www.fs.usda.gov/rds/archive/products/RDS-2013-0009.4/RDS-2013-0009.4_Data_Format5_SQLite.zip'
fires_file = 'RDS-2013-0009.4_Data_Format5_SQLite.zip'

## Run queries against the database

In [47]:
# Download zip file

import sqlite3
import zipfile
import requests
import os

# Instructor's note: Everything that I'm about to say is outside the scope of this class; however, it is
# very useful for you to know as you'll most certainly work with data from different sources and you have
# to know how to get around obstacles.

# It appears that the USDA website restricts the ability download files directly using Python's request
# library. As such, you have to make the website think that you are downloading the file from a web browser.
# To do this, you must define the "header" for the request. I've done this for you here.

# The header is metadata sent to the website that gives it some information about the source making the
# request. The header information below was obtained by going to the USDA page and looking in Chrome's
# DevTools window under the network tab. Here, you'll be able to see the requests made by the browser to
# the website. To see this header information, right click the request in the network tab, then select copy,
# and click "Copy as cURL" either PowerShell or Bash. The point is that you'll have a command copied to your
# clipboard that you can execute in your command console (either through PowerShell or Bash, depending on your
# choice). To get it into the format you see below, you'll have to modify the header information manually.
# Again, this is outside the scope of this class and you're not expected to know this here, but I highly
# recommend that you learn about web scraping and all the techniques involved.

headers = { 'origin': 'https://www.fs.usda.gov'
          , 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36'}

print('Requesting file')

with requests.get(fires_url, stream = True, headers = headers, timeout = 15) as response:
    response.raise_for_status()
    total_size = int(response.headers.get('content-length', 0))

    print(f'Total file size is {total_size:,} bytes')
    
    chunk_sum  = 0
    
    print('Writing to file')
    
    with open(fires_file, 'wb') as f:
        for chunk in response.iter_content(chunk_size = 4096):
            chunk_sum += f.write(chunk)
            download_pct = (chunk_sum / total_size) * 100
            print(f'\rPercent done: {download_pct:.2f}%', end = '')
    
    print(f'. Done.')

Requesting file
Total file size is 173,776,420 bytes
Writing to file
Percent done: 100.00%. Done.


In [48]:
# Unzip file
with zipfile.ZipFile(fires_file) as zip:
    zip.extractall()

In [49]:
# Load to database
connect = sqlite3.connect('Data/FPA_FOD_20170508.sqlite')
cursor  = connect.cursor()

In [50]:
# Print the list of tables in this database
query = "SELECT name FROM sqlite_master WHERE type='table';"
cursor.execute(query)
cursor.fetchall()

[('spatial_ref_sys',),
 ('spatialite_history',),
 ('sqlite_sequence',),
 ('geometry_columns',),
 ('spatial_ref_sys_aux',),
 ('views_geometry_columns',),
 ('virts_geometry_columns',),
 ('geometry_columns_statistics',),
 ('views_geometry_columns_statistics',),
 ('virts_geometry_columns_statistics',),
 ('geometry_columns_field_infos',),
 ('views_geometry_columns_field_infos',),
 ('virts_geometry_columns_field_infos',),
 ('geometry_columns_time',),
 ('geometry_columns_auth',),
 ('views_geometry_columns_auth',),
 ('virts_geometry_columns_auth',),
 ('sql_statements_log',),
 ('SpatialIndex',),
 ('ElementaryGeometries',),
 ('KNN',),
 ('Fires',),
 ('idx_Fires_Shape',),
 ('idx_Fires_Shape_node',),
 ('idx_Fires_Shape_rowid',),
 ('idx_Fires_Shape_parent',),
 ('NWCG_UnitIDActive_20170109',)]

In [68]:
query = "" # your query goes here
cursor.execute(query)

## Read the results of a SQL query into a DataFrame

In [89]:
# you should know how to do this from your textbook.
query = "SELECT * FROM Fires"
fires = pd.read_sql_query(query,connect)

In [90]:
fires.head()

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,...,A,40.036944,-121.005833,5.0,USFS,CA,63,063,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.933056,-120.404444,5.0,USFS,CA,61,061,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17,017,El Dorado,b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.913333,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.933056,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...


In [91]:
fires.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1880465 entries, 0 to 1880464
Data columns (total 39 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   OBJECTID                    int64  
 1   FOD_ID                      int64  
 2   FPA_ID                      object 
 3   SOURCE_SYSTEM_TYPE          object 
 4   SOURCE_SYSTEM               object 
 5   NWCG_REPORTING_AGENCY       object 
 6   NWCG_REPORTING_UNIT_ID      object 
 7   NWCG_REPORTING_UNIT_NAME    object 
 8   SOURCE_REPORTING_UNIT       object 
 9   SOURCE_REPORTING_UNIT_NAME  object 
 10  LOCAL_FIRE_REPORT_ID        object 
 11  LOCAL_INCIDENT_ID           object 
 12  FIRE_CODE                   object 
 13  FIRE_NAME                   object 
 14  ICS_209_INCIDENT_NUMBER     object 
 15  ICS_209_NAME                object 
 16  MTBS_ID                     object 
 17  MTBS_FIRE_NAME              object 
 18  COMPLEX_NAME                object 
 19  FIRE_YEAR            

## Close Database File

In [ ]:
cursor.close()
connect.close()